# Calculator Load and Check
+ Load a calculator template
+ Load in dummy values
+ Check the outputs are as expected

## Objects
### Calculation template
+ Spreadsheet template with all the relationships between fields expressed. Libreoffice is used to execute this, excel can also be used.

### Measurement Module
+ Someway to take the names of fields and translate these into measurement commands. In this case the names match a modbus lookup table.

### Calculation Output
+ Filled in calculation template with all fields calculated

### Test matrix
+ Defines the names of the fields to check
+ Has the operation which is in a lookup table. We're only using simple operations (min/max or equal) more complex relationships should be expressed in the calculation template

In [1]:
import pandas
import pyexcel, openpyxl
import os, sys, importlib
sys.path.append("../")
import pyetest
importlib.reload(pyetest)

<module 'pyetest' from '/home/simon/projects/pyetest/examples/../pyetest.py'>

In [2]:
template_file = "./calculator-sheet-blank-template.xlsx"
measured_data_file = "./dummy-values.xlsx"
test_matrix_file = "./value-limits.xlsx"
out_file = "filled-data-table.xlsx"

workbook = openpyxl.open(template_file)
dummy_values_df = pandas.read_excel(measured_data_file)
compiled_wb = pyetest.compile_template(workbook, data=dummy_values_df)
processed_wb = pyetest.reload_spreadsheet(compiled_wb)
processed_wb.save(out_file)

convert /tmp/tmpz8iol7mr/pyetest.xlsx -> /tmp/tmpux6k9ra4/pyetest.xlsx using filter : Calc Office Open XML


In [13]:
# Tests
test_results_file = "test-results.xlsx"
processed_df = pandas.read_excel(out_file)
values_matrix_df = pandas.read_excel(test_matrix_file).dropna(subset=["operation"])
test_result_df = pyetest.run_text_matrix(processed_df, values_matrix_df)
tests_passed = all(test_result_df["passes"])
test_result_df["operation"] = [f"'{op}" for op in test_result_df["operation"]]
test_result_df.to_excel(test_results_file)

In [12]:
for row in processed_wb.active.iter_rows(min_row=1, values_only=False):
    print(row[1].coordinate, row[0].value, "\t\t", row[1].value)

B1 name 		 value
B2 version 		 0.8.X
B3 firmware_version 		 0.1.8
B4 compile_time 		 09:09:00
B5 compile_date 		 September 9th, 2023
B6 slave_address 		 246
B7 p3_3_micro_volts 		 3260000
B8 p3_3_reading 		 2024
B9 +3.3GAIN 		 0.5
B10 +3.3OFFSET 		 0
B11 +3.3V 		 3.261328125
B12 +3.3ERROR 		 86.4111328125
B13 +3.3COMPARISONERROR 		 0.0407400306748462
B14 p23_micro_volts 		 23000000
B15 p23_reading 		 1200
B16 +23GAIN 		 0.04
B17 +23OFFSET 		 0
B18 +23V 		 24.169921875
B19 +23ERROR 		 3.73357027896994
B20 +23COMPARISONERROR 		 5.0866168478261
B21 p5_micro_volts 		 5000000
B22 p5_reading 		 2500
B23 +5OFFSET 		 0
B24 +5GAIN 		 2.5
B25 +5 		 5.035400390625
B26 +5ERROR 		 0.7080078125
B27 +5COMPARISONERROR 		 0.7080078125
B28 fault_status 		 9
B29 +5 Fault 		 1
B30 +24 Fault 		 0
B31 Error Code 2 		 0
B32 DAQ Fault 		 1
B33 Error Code 4 		 0
B34 MW Fault 		 0
B35 Error Code 6 		 0
B36 Error Code 7 		 0
B37 Visible Fault 		 0
B38 Temperature Fault 		 0
B39 Moisture Fault 		 0
B40 Watchdog F

In [10]:
print(processed_df)

                   name                value unit        type
0               version                0.8.X  NaN    measured
1      firmware_version                0.1.8  NaN    measured
2          compile_time             09:09:00  NaN    measured
3          compile_date  September 9th, 2023  NaN    measured
4         slave_address                  246  NaN    measured
5      p3_3_micro_volts              3260000   uV    measured
6          p3_3_reading                 2024  ADC    measured
7              +3.3GAIN                  0.5  NaN       fixed
8            +3.3OFFSET                    0    V       fixed
9                 +3.3V             3.261328    V  calculated
10            +3.3ERROR            86.411133    %  calculated
11  +3.3COMPARISONERROR              0.04074    %  calculated
12      p23_micro_volts             23000000   uV    measured
13          p23_reading                 1200  ADC    measured
14              +23GAIN                 0.04  NaN       fixed
15      

In [7]:
print(tests_passed)
print(test_result_df)

False
                 name       expected value unit         min         max  \
0             version                0.8.X  NaN         NaN         NaN   
1    firmware_version                0.1.8  NaN         NaN         NaN   
2        compile_time             09:09:00  NaN         NaN         NaN   
3        compile_date  September 9th, 2023  NaN         NaN         NaN   
4       slave_address                  246  NaN         NaN         NaN   
5    p3_3_micro_volts              3300000   uV   3200000.0   3400000.0   
7           +3.3ERROR                    0    %         0.0         3.0   
8     p23_micro_volts             23000000   uV  18000000.0  30000000.0   
10           +23ERROR                    0    %         0.0         3.0   
11     p5_micro_volts              5000000   uV   4900000.0   5100000.0   
13            +5ERROR                    0    %         0.0         3.0   
15           +5 Fault                    0  NaN         NaN         NaN   
16          +24 Fau